## WICCI Downscaled Data: Precip - 30 year - seasonal - Historic (CMIP6 Source)

Objectives
* aggregate data for precip, e.g. mean, standard deviation, totals, for WICCI's historic climate data
* run for 30 year model window, e.g. 1981-2010 for 1995 average (matches data on WICCI site)
* calculate averages by seasons (Winter: DJF, Spring: MAM, Summer: JJA, Fall: SON)
* create new netcdf file(s) for aggregate data for 30-year timeframe
* calculate intermodel standard deviation differently -- find average of each model within the current time window and
  then calculate standard deviation across models

Eric Compas, compase@uww.edu 11/17/2021, 1/19/2022, 1/25/2022, 2/28/2022, 12/4/2024, 1/31/2025

In [1]:
import netCDF4
import numpy as np
import os
import datetime
import gc
from netCDF4 import Dataset,num2date,date2num
import warnings
import logging
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Which folder are we getting the downscaled climate data from? Needs to be the data from WICCI Climate Working Group with folder structure as provided. This level should contain the 'rcp45' and 'rcp85' folders to work correctly.
base_folder = "X:\CMIP6_Data"
if not os.path.isdir(base_folder):
    print("Base folder not valid")

In [3]:
# Folder to store the resulting netCDF files in. Can be anywhere on your file system. Note that this script will not overwrite existing files.
#out_folder = "Z:/Climate_Data/aggregate_files"
out_folder = r"C:\Users\compase\Dropbox\Spring_2025\WICCI\CMIP6_WICCI_Output"
#out_folder  = "/Users/ericcompas/Climate_Data/output"
if not os.path.isdir(out_folder):
    print("Out folder not valid")

In [ ]:
# Define seasons and their date ranges
seasons = [
    ('Winter', ([335,366], [0,59])),    # Dec-Feb
    ('Spring', ([60,151], None)),       # Mar-May
    ('Summer', ([152,243], None)),      # Jun-Aug
    ('Fall', ([244,334], None))         # Sep-Nov
]

def process_season_chunk(data_array, season_range1, season_range2=None):
    """Process a chunk of data for a given season"""
    if season_range2 is None:
        # Regular season
        season_data = data_array[season_range1[0]:season_range1[1]+1, :, :]
    else:
        # Winter season (spans year boundary)
        part1 = data_array[season_range1[0]:season_range1[1], :, :]
        part2 = data_array[season_range2[0]:season_range2[1]+1, :, :]
        season_data = np.ma.concatenate([part1, part2], axis=1)
    return season_data

def calculate_seasonal_stats(data_array, mask, threshold=None, comparison=None):
    """Calculate seasonal statistics for a data array"""
    if threshold is not None and comparison is not None:
        if comparison == 'gt':
            count = (data_array >= threshold).sum(axis=(0,1))
        elif comparison == 'lt':
            count = (data_array <= threshold).sum(axis=(0,1))
        elif comparison == 'eq':
            count = (data_array == threshold).sum(axis=(0,1))
        num_days = data_array.shape[1]
        num_samples = data_array.shape[0] * num_days
        return np.ma.masked_array((count / num_samples) * num_days, mask)
    else:
        return np.ma.masked_array(np.ma.sum(data_array, axis=(0,1)), mask)

# The BIG LOOP with memory-efficient processing
models = ["historical"]
for m in models:
    print("Processing climate scenario "+m)
    
    GCMs = os.listdir(os.path.join(base_folder,m))
    GCMs = GCMs[:5]  # for testing only
    
    for y in range(1995,1996,30):    
        print("  Processing base year "+str(y))
        
        # Initialize seasonal arrays
        prcp_seasons_total = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_std = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_gt1 = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_gt2 = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_gt3 = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_gt4 = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_gt5 = np.ma.empty([4, 89, 91], dtype=np.float32)
        prcp_seasons_norain = np.ma.empty([4, 89, 91], dtype=np.float32)
        
        # Process each GCM
        for gcm_idx, gcm in enumerate(GCMs):
            print(f"    Processing GCM {gcm} ({gcm_idx+1}/{len(GCMs)})")
            
            # Process each season
            for season_idx, (season_name, (range1, range2)) in enumerate(seasons):
                print(f"      Processing {season_name}")
                
                # Initialize seasonal accumulators
                prcp_acc = []
                
                # Process each year
                for yr in range(y-14, y+16):
                    for r in ["01","02","03"]:
                        nf = os.path.join(base_folder,m,gcm,"r1i1p1",f"prcp_{r}_{yr}.nc")
                        print(f"      Opening {nf}...")
                        try:
                            with netCDF4.Dataset(nf) as n:
                                # Convert to inches and handle leap years
                                prcp = n.variables['prcp'][:] * 0.0393701
                                
                                if prcp.shape[0] == 366:
                                    # Average Feb 28-29
                                    prcp_leap = prcp[58:60,:,:]
                                    prcp_mean_leap = np.ma.mean(prcp_leap,axis=0)
                                    prcp = np.ma.concatenate([
                                        prcp[0:58],
                                        prcp_mean_leap[np.newaxis],
                                        prcp[60:]
                                    ])
                                
                                # Get seasonal chunk
                                prcp_season = process_season_chunk(prcp, range1, range2)
                                prcp_acc.append(prcp_season)
                                
                                # Save mask from first file
                                if len(prcp_acc) == 1:
                                    mask = np.ma.getmask(prcp[0])
                        except BaseException:
                            logging.exception(f"        Can't locate netCDF file for year: {yr} or error processing year")
                            continue
                        
                        # Free memory
                        gc.collect()
                
                # Calculate seasonal statistics
                if len(prcp_acc) > 0:
                    prcp_season_data = np.ma.stack(prcp_acc)
                    
                    # Calculate total precipitation
                    prcp_seasons_total[season_idx] = calculate_seasonal_stats(prcp_season_data, mask)
                    
                    # Calculate standard deviation
                    prcp_seasons_std[season_idx] = np.ma.std(prcp_season_data, axis=(0,1))
                    
                    # Calculate extremes
                    prcp_seasons_gt1[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 1, 'gt')
                    prcp_seasons_gt2[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 2, 'gt')
                    prcp_seasons_gt3[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 3, 'gt')
                    prcp_seasons_gt4[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 4, 'gt')
                    prcp_seasons_gt5[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 5, 'gt')
                    prcp_seasons_norain[season_idx] = calculate_seasonal_stats(prcp_season_data, mask, 0, 'eq')
                    
                    # Free memory
                    del prcp_season_data
                    gc.collect()
        
        # Write results to netCDF file
        filename = "prcp_historical_1981-2010_seasonal.nc"
        newfile = os.path.join(out_folder,filename)
        with Dataset(newfile, mode='w', format='NETCDF4_CLASSIC') as ncfile:
            # Create dimensions
            lat_dim = ncfile.createDimension('lat', 89)
            lon_dim = ncfile.createDimension('lon', 91)
            time_dim = ncfile.createDimension('time', None)
            
            # Add metadata
            ncfile.title = f'Aggregate seasonal precipitation values for WICCI downscaled climate data (CMIP6) for all GCMs for {m} and 30-year window around year {y}'
            ncfile.subtitle = "Data source: UW-Madison WICCI; Data aggregation: Eric Compas, compase@uww.edu"
            
            # Create variables
            lat = ncfile.createVariable('lat', np.float32, ('lat',))
            lon = ncfile.createVariable('lon', np.float32, ('lon',))
            time = ncfile.createVariable('time', np.float32, ('time',))
            
            # Add variable metadata
            lat.units = 'degrees_north'
            lat.long_name = 'latitude'
            lon.units = 'degrees_east'
            lon.long_name = 'longitude'
            time.units = f'days since {y}-01-01'
            time.long_name = 'time'
            
            # Create precipitation variables
            var_defs = [
                ('prcp_total', prcp_seasons_total, 'total precipitation (inches) per season'),
                ('prcp_std', prcp_seasons_std, 'standard deviation of daily precipitation (inches) per season'),
                ('prcp_gt1', prcp_seasons_gt1, 'estimated days with >= 1 inch (25.4mm) precipitation per season'),
                ('prcp_gt2', prcp_seasons_gt2, 'estimated days with >= 2 inches (50.8mm) precipitation per season'),
                ('prcp_gt3', prcp_seasons_gt3, 'estimated days with >= 3 inches (76.2mm) precipitation per season'),
                ('prcp_gt4', prcp_seasons_gt4, 'estimated days with >= 4 inches (101.6mm) precipitation per season'),
                ('prcp_gt5', prcp_seasons_gt5, 'estimated days with >= 5 inches (127.0mm) precipitation per season'),
                ('prcp_norain', prcp_seasons_norain, 'estimated days with no precipitation per season')
            ]
            
            for name, data, desc in var_defs:
                var = ncfile.createVariable(name, np.float32, ('time','lat','lon'))
                var.units = 'inches' if 'inches' in desc else 'days'
                var.standard_name = desc
                var.missing_value = -32768
                var[:] = data
            
            # Write coordinates
            lat[:] = n.variables['lat'][:]
            lon[:] = n.variables['lon'][:]
            
            # Write time - middle of each season
            season_midpoints = [
                datetime.datetime(y,1,15,0),  # Winter (Jan 15)
                datetime.datetime(y,4,15,0),  # Spring (Apr 15)
                datetime.datetime(y,7,15,0),  # Summer (Jul 15)
                datetime.datetime(y,10,15,0)  # Fall (Oct 15)
            ]
            time[:] = date2num(season_midpoints, time.units)
        
        print(f"Wrote: {filename}")

ERROR:root:        Can't locate netCDF file for year: 1981
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_season = process_season_chunk(prcp, range1, range2)
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 18, in process_season_chunk
    season_data = np.ma.concatenate([part1, part2], axis=1)
  File "c:\Users\compase\AppData\Local\anaconda3\envs\climate\lib\site-packages\numpy\ma\core.py", line 7257, in concatenate
    d = np.concatenate([getdata(a) for a in arrays], axis)
ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30 and the array at index 1 has size 60
ERROR:root:        Can't locate netCDF file for year: 1981
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_seaso

Processing climate scenario historical
  Processing base year 1995
    Processing GCM ACCESS-CM2 (1/5)
      Processing Winter
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_01_1981.nc...
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_02_1981.nc...
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_03_1981.nc...


ERROR:root:        Can't locate netCDF file for year: 1982
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_season = process_season_chunk(prcp, range1, range2)
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 18, in process_season_chunk
    season_data = np.ma.concatenate([part1, part2], axis=1)
  File "c:\Users\compase\AppData\Local\anaconda3\envs\climate\lib\site-packages\numpy\ma\core.py", line 7257, in concatenate
    d = np.concatenate([getdata(a) for a in arrays], axis)
ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30 and the array at index 1 has size 60
ERROR:root:        Can't locate netCDF file for year: 1982
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_seaso

      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_01_1982.nc...
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_02_1982.nc...
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_03_1982.nc...
      Opening X:\CMIP6_Data\historical\ACCESS-CM2\r1i1p1\prcp_01_1983.nc...


ERROR:root:        Can't locate netCDF file for year: 1983
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_season = process_season_chunk(prcp, range1, range2)
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 18, in process_season_chunk
    season_data = np.ma.concatenate([part1, part2], axis=1)
  File "c:\Users\compase\AppData\Local\anaconda3\envs\climate\lib\site-packages\numpy\ma\core.py", line 7257, in concatenate
    d = np.concatenate([getdata(a) for a in arrays], axis)
ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30 and the array at index 1 has size 60
ERROR:root:        Can't locate netCDF file for year: 1983
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_seaso

...

      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_2008.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_2008.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_2009.nc...


ERROR:root:        Can't locate netCDF file for year: 2009
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_season = process_season_chunk(prcp, range1, range2)
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 18, in process_season_chunk
    season_data = np.ma.concatenate([part1, part2], axis=1)
  File "c:\Users\compase\AppData\Local\anaconda3\envs\climate\lib\site-packages\numpy\ma\core.py", line 7257, in concatenate
    d = np.concatenate([getdata(a) for a in arrays], axis)
ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30 and the array at index 1 has size 60
ERROR:root:        Can't locate netCDF file for year: 2009
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_seaso

      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_2009.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_2009.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_2010.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_2010.nc...


ERROR:root:        Can't locate netCDF file for year: 2010
Traceback (most recent call last):
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 89, in <module>
    prcp_season = process_season_chunk(prcp, range1, range2)
  File "C:\Users\compase\AppData\Local\Temp\ipykernel_26060\2698105142.py", line 18, in process_season_chunk
    season_data = np.ma.concatenate([part1, part2], axis=1)
  File "c:\Users\compase\AppData\Local\anaconda3\envs\climate\lib\site-packages\numpy\ma\core.py", line 7257, in concatenate
    d = np.concatenate([getdata(a) for a in arrays], axis)
ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 30 and the array at index 1 has size 60


      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_2010.nc...
      Processing Spring
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_1981.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_1981.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_1981.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_1982.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_1982.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_1982.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_1983.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_1983.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_1983.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_01_1984.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_02_1984.nc...
      Opening X:\CMIP6_Data\historical\CanESM5\r1i1p1\prcp_03_1984.nc...
      Opening X:\CMIP6_Data